# Oscilloscope Capture Example

This notebook demonstrates the workflow for capturing, saving, loading, decoding, and plotting oscilloscope data.

In [ ]:
# Imports
import serial
import time
import matplotlib.pyplot as plt

from analysis_lib import KeysightInfiniiVisionMSO, DEFAULT_LABELS
from analysis_lib.decoder import RawDecoder, BrotherSerialDecoder
from analysis_lib.plotting import plot_digital, plot_digital_normalized
from analysis_lib.capture import Capture

In [ ]:
# Setup oscilloscope connection and configuration
ADDRESS = "USB0::2391::5925::MY49110266::INSTR"

scope = KeysightInfiniiVisionMSO(ADDRESS)

# Configure digital channels
scope.setup_digital_channels(
    channels=range(6),
    threshold=2.5,
    time_scale=1.5e-4,
)

# Set channel labels
for i, label in enumerate(DEFAULT_LABELS):
    scope.set_digital_label(i, label)

# Configure trigger
scope.setup_digital_trigger(channel=4, slope="NEGative")
scope.set_trigger_delay(4.5e-4)

In [ ]:
# Capture data from oscilloscope
# Arm the trigger and send signal to device
scope.arm_trigger()
time.sleep(0.2)

# Example: Send command via serial to trigger capture
with serial.Serial(
    port='/dev/ttyUSB0',
    baudrate=9600,
    bytesize=serial.EIGHTBITS,
    stopbits=serial.STOPBITS_ONE,
    parity=serial.PARITY_NONE,
    rtscts=True,
    timeout=1
) as con:
    con.write(bytes([0x55]))  # Send command to device
    time.sleep(1)

# Read captured data from oscilloscope
time_data, packed, channel_data = scope.read_current_pod_data(pod=1, mode='RAW')

print(f"Captured {len(time_data)} samples")

In [ ]:
# Create and save capture with metadata
capture = Capture.from_scope_data(
    time_data, packed, channel_data,
    typewriter="TYPEWRITER",
    interface="INTERFACE",
    name="EXAMPLE_CAPTURE",
    info="Example capture description",
    keyboard_setting=1,
    interface_dip_switches=0b111111111111111
)

# Display capture info
print(capture.get_info())

# Save to file
filepath = capture.save(directory="./signal_captures/EXAMPLE")
print(f"\nSaved to: {filepath}")

In [ ]:
# Load capture from disc
filepath = "./signal_captures/EXAMPLE/TYPEWRITER_INTERFACE_EXAMPLE_CAPTURE.pkl"
capture = Capture.load(filepath)

# Extract data from loaded capture
time_data = capture.time_data
channel_data = capture.channel_data

print(f"Loaded capture: {capture.name}")
print(capture.get_info())

In [ ]:
# Decode raw signals into frames and bytes
raw_decoder = RawDecoder(channel_data, time_data)

# Decode SI (Serial In) and SO (Serial Out) channels
raw_si = raw_decoder.decode(data_ch=0)
raw_so = raw_decoder.decode(data_ch=1)

print(f"RAW-SI: {raw_si}")
print(f"RAW-SO: {raw_so}")

# Decode to Brother Serial Protocol
decoded_si = BrotherSerialDecoder(raw_si).decode()
decoded_so = BrotherSerialDecoder(raw_so).decode()

print(f"\nDEC-SI: {decoded_si}")
print(f"DEC-SO: {decoded_so}")

In [ ]:
# Plot captured signals
# Create annotations from decoded data
annotations = (
    decoded_si.to_annotations(include_bits=True) + 
    decoded_so.to_annotations(include_bits=True)
)

# Plot raw digital signals
plot_digital(time_data, channel_data)

# Plot normalized view with annotations
plot_digital_normalized(time_data, channel_data, annotations=annotations)

In [ ]:
scope.close()